In [155]:
import os
import math

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from PIL import Image

In [127]:
tf.test.is_gpu_available()

True

In [128]:
gitw_path = "/espace/GITW"

In [129]:
classes = os.listdir(gitw_path)

In [130]:
directories = []
labels = []
for d_name in os.listdir(gitw_path):
    for sub_name in os.listdir(os.path.join(gitw_path, d_name)):
        directories.append(os.path.join(d_name, sub_name, "patches"))
        labels.append(int(classes.index(d_name)))

In [131]:
selected_indexes = np.random.choice([0, 1], p=[0.8, 0.2], size=(len(directories),))
train_dir, train_labels = list(np.array(directories)[selected_indexes == 0]), list(np.array(labels)[selected_indexes == 0])
test_dir, test_labels = list(np.array(directories)[selected_indexes == 1]), list(np.array(labels)[selected_indexes == 1])

In [137]:
pd.DataFrame({"directories": train_dir, "labels": train_labels}).to_csv("traindata.csv", index=False)
pd.DataFrame({"directories": test_dir, "labels": test_labels}).to_csv("testdata.csv", index=False)

In [163]:
class GITWDataset(keras.utils.Sequence):
    
    def __init__(self, path, directories, labels, batch_size=32, max_frame=10):
        self.X = directories
        self.Y = keras.utils.to_categorical(labels)
        self.batch_size = batch_size
        self.max_frame = max_frame
        self.path = path
        
    def __len__(self):
        return math.ceil(len(self.X) / self.batch_size)
    
    def __getitem__(self, idx):
        files_x = self.X[idx*self.batch_size:(idx+1)*self.batch_size]
        batch_y = self.Y[idx*self.batch_size:(idx+1)*self.batch_size]
        batch_x = []
        for file in files_x:
            frames_path = os.path.join(self.path, file)
            frames = [np.array(Image.open(os.path.join(frames_path, frame)).convert("RGB").getdata()).reshape((227,227,3)) / 255 for frame in os.listdir(frames_path) if os.path.isfile(os.path.join(frames_path, frame))]
            frames = np.stack(frames)
            batch_x.append(frames[:self.max_frame])
        return np.stack(batch_x), batch_y

In [164]:
train_data = pd.read_csv("traindata.csv")
test_data = pd.read_csv("testdata.csv")

trainset = GITWDataset(gitw_path, train_data.directories, train_data.labels)
testset = GITWDataset(gitw_path, test_data.directories, test_data.labels)

In [165]:
m = keras.models.Sequential([
    keras.layers.Flatten(input_shape=(10, 227, 227, 3)),
    keras.layers.Dense(16)
])
m.compile(loss="categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])

In [ ]:
m.fit_generator(trainset, len(trainset)*32, epochs=1)

Epoch 1/1


Exception in thread Thread-9:
Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.5/threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "/autofs/unityaccount/ens/DeepLearning/python3/tensorflow/lib/python3.5/site-packages/tensorflow/python/keras/_impl/keras/utils/data_utils.py", line 511, in _run
    self.sequence.on_epoch_end()
  File "/autofs/unityaccount/ens/DeepLearning/python3/tensorflow/lib/python3.5/site-packages/tensorflow/python/keras/_impl/keras/utils/data_utils.py", line 375, in on_epoch_end
    raise NotImplementedError
NotImplementedError

